In [1]:
from login_using_selenium import login_using_selenium
from telegram_bot import telegram_bot_sendtext
from ticker_service import Ticker_class
from orders import Class_Orders
from functions_collections import Kite_functions
from config import kite_username, kite_password, kite_pin
from kite_ext_new import KiteExt_new
from redis import Redis
import datetime
import time
import traceback
import subprocess
from straddle_strategy2 import straddles
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler('main.log')
file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(message)s'))
logger.addHandler(file_handler)

redis_server = subprocess.Popen('redis-server')

redis_obj = Redis(host='127.0.0.1', port=6379, decode_responses=True)

In [2]:
telegram_bot_sendtext('Starting Algo...')

kite = KiteExt_new()

#selenium_login_status = login_using_selenium()
selenium_login_status = True

if selenium_login_status:
    with open('enctoken.txt', 'r') as enctoken_file:
        enctoken = enctoken_file.read()
        kite.login_using_enctoken(kite_username, enctoken, None)

else:
    logger.info("Logging in using Kiteext")
    telegram_bot_sendtext("Logging in using Kiteext")
    kite.login_with_credentials(userid=kite_username, password=kite_password, pin=kite_pin)

In [3]:
ticker = Ticker_class(kite=kite, tokens = [256265, 260105])
kite_func = Kite_functions(kite)
orders_obj = Class_Orders(kite = kite, redis_obj = redis_obj, kite_functions = kite_func)

#straddles_obj = straddles(kite=kite, kite_func= kite_func, orders_obj = orders_obj, margin=1)
straddles_obj = straddles(kite=kite, kite_func= kite_func, orders_obj = orders_obj, redis_obj = redis_obj, ticker = ticker, margin=1)

In [4]:
#ticker.tokens = straddles_obj.nf_bnf_option_tokens
ticker.start_ticker()

In [ ]:
current_dt = datetime.datetime.now()
error_count = 0
while current_dt.hour <= 15 and not (current_dt.hour >= 15 and current_dt.minute >= 30):
    try:
        current_dt = datetime.datetime.now()
        if current_dt.hour == 15 and current_dt.minute >= 11:
            #Exit from all open positions at 3:15.
            # kite_day_positions_list = kite_func.get_positions_list()['net']
            # for each_position in kite_day_positions_list:
            #     try:
            #         if each_position['quantity'] != 0 and each_position['product'] == 'MIS':
            #             stock = each_position['tradingsymbol']
            #             exit_quantity = abs(each_position['quantity'])
            #             exit_type = "sell" if each_position['quantity'] > 0 else "buy"
            #             if exit_quantity > 0:
            #                 orders_obj.exit_position(stock, exit_type, exit_quantity)

            #                 telegram_bot_sendtext(f"Exiting from {stock} at 3:15")
            #     except Exception as e:
            #         print("Unexpected error while exiting positions at 3:15. Error: "+str(e))
            #         traceback.print_exc()
            break

        straddles_obj.main()
        time.sleep(2)


    except Exception as e:
        logger.exception("Unexpected error in main while loop. Error: "+str(e))
        traceback.print_exc()
        error_count += 1
        if error_count >100:
            break

    except KeyboardInterrupt:
        logger.exception('\nKeyboard exception received. Exiting.')
        break

logger.info('Stopping Websocket')
ticker.kws.stop()
redis_server.kill()

In [ ]:
ticker.kws.stop()

Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


In [6]:
ticker.tokens

[256265, 260105]

In [3]:
redis_server.kill()

In [6]:
kite_func.get_symbol_token('NSE:NIFTY 50'), kite_func.get_symbol_token('NSE:NIFTY BANK')

(256265, 260105)

In [16]:
ticker.ticker_dict[13247234]

[]

In [7]:
ticker.websocket_is_open

False

In [8]:
ticker.kws.is_connected()

True

In [12]:
redis_obj.get(11019266)

'189.8'

In [9]:
kite_func.get_options_symbol_and_token('BANKNIFTY', 32500, 'PE')

('BANKNIFTY2231032500PE', 13247234)

In [15]:
ticker.depth_dict[14662658]

{'buy': [{'quantity': 50, 'price': 597.15, 'orders': 2},
  {'quantity': 50, 'price': 596.95, 'orders': 1},
  {'quantity': 50, 'price': 596.9, 'orders': 1},
  {'quantity': 50, 'price': 596.8, 'orders': 1},
  {'quantity': 50, 'price': 596.7, 'orders': 1}],
 'sell': [{'quantity': 50, 'price': 597.75, 'orders': 2},
  {'quantity': 50, 'price': 597.8, 'orders': 1},
  {'quantity': 50, 'price': 597.85, 'orders': 2},
  {'quantity': 575, 'price': 597.95, 'orders': 3},
  {'quantity': 425, 'price': 598.0, 'orders': 2}]}

In [16]:
ticker.depth_dict[14662658]

{'buy': [{'quantity': 25, 'price': 643.6, 'orders': 1},
  {'quantity': 25, 'price': 643.45, 'orders': 1},
  {'quantity': 25, 'price': 643.4, 'orders': 1},
  {'quantity': 25, 'price': 643.35, 'orders': 1},
  {'quantity': 25, 'price': 643.3, 'orders': 1}],
 'sell': [{'quantity': 25, 'price': 644.45, 'orders': 1},
  {'quantity': 25, 'price': 644.5, 'orders': 1},
  {'quantity': 25, 'price': 644.55, 'orders': 1},
  {'quantity': 200, 'price': 644.7, 'orders': 1},
  {'quantity': 25, 'price': 644.75, 'orders': 1}]}

In [3]:
kite_func = Kite_functions(kite)
#orders_obj = Class_Orders(kite = kite, ticker = ticker, kite_functions = kite_func)
straddles_obj = straddles(kite=kite, kite_func= kite_func, orders_obj = None, margin=1)

[13419266, 13419522, 13360642, 13361922, 13419778, 13420034, 13362178, 13366018, 13420290, 13420546, 13366274, 13367554, 13420802, 13421058, 13367810, 13369602, 13421314, 13421570, 13369858, 13371138, 13421826, 13422082, 13371394, 13371650, 13422338, 13422594, 13371906, 13372162, 13422850, 13423106, 13372418, 13372674, 13423362, 13423618, 13372930, 13391874, 13423874, 13424130, 13392130, 13392386, 13424386, 13424642, 13392642, 13392898, 13424898, 13425154, 13393154, 13393410, 13425410, 13425666, 13393666, 13393922, 13425922, 13426178, 13394178, 13394434, 13426434, 13426690, 13394690, 13394946, 13426946, 13427202, 13395202, 13395458, 13427458, 13427714, 13395714, 13395970, 13427970, 13428226, 13396226, 13396482, 13428482, 13428738, 13396738, 13396994, 13428994, 13429250, 13397250, 13397506, 13418754, 13419010, 13360130, 13360386, 13418242, 13418498, 13359618, 13359874, 13417730, 13417986, 13359106, 13359362, 13417218, 13417474, 13358594, 13358850, 13416706, 13416962, 13358082, 13358338,

In [5]:
kite.ltp('MCX:CRUDEOIL22MARFUT')

{'MCX:CRUDEOIL22MARFUT': {'instrument_token': 59736071, 'last_price': 9596}}

In [3]:
ticker = Ticker_class(kite=kite, tokens = [59736071, 60451079])

In [17]:
256265
ticker = Ticker_class(kite=kite, tokens = [59736071])

In [4]:
ticker.start_ticker()

60451079 not in dict.
59736071 not in dict.


In [7]:
import redis
r = redis.Redis(host='127.0.0.1', port=6379, decode_responses=True)

In [6]:
r = redis_obj

In [26]:
eval(r.get(59736071))

7938.0

In [7]:
eval(r.get('59736071'+'_depth'))#['buy'][1]['price']

{'buy': [{'quantity': 0, 'price': 0.0, 'orders': 0},
  {'quantity': 0, 'price': 0.0, 'orders': 0},
  {'quantity': 0, 'price': 0.0, 'orders': 0},
  {'quantity': 0, 'price': 0.0, 'orders': 0},
  {'quantity': 0, 'price': 0.0, 'orders': 0}],
 'sell': [{'quantity': 790, 'price': 8589.0, 'orders': 198},
  {'quantity': 6, 'price': 8590.0, 'orders': 6},
  {'quantity': 1, 'price': 8591.0, 'orders': 1},
  {'quantity': 1, 'price': 8593.0, 'orders': 1},
  {'quantity': 1, 'price': 8595.0, 'orders': 1}]}

In [28]:
#r.lrange('59736071'+'_data', 0, -1)
ltp_data = list(map(eval, r.lrange('59736071'+'_data', 0, -1)))
ltp_data[:-5]

[[datetime.datetime(2022, 3, 8, 19, 53, 54), 9588.0, 75882],
 [datetime.datetime(2022, 3, 8, 19, 53, 57), 9585.0, 75883],
 [datetime.datetime(2022, 3, 8, 19, 53, 58), 9585.0, 75885],
 [datetime.datetime(2022, 3, 8, 19, 53, 59), 9588.0, 75891],
 [datetime.datetime(2022, 3, 8, 19, 54, 1), 9602.0, 75914],
 [datetime.datetime(2022, 3, 8, 19, 54, 2), 9602.0, 75915],
 [datetime.datetime(2022, 3, 8, 19, 54, 2), 9604.0, 75916],
 [datetime.datetime(2022, 3, 8, 19, 54, 4), 9600.0, 75917],
 [datetime.datetime(2022, 3, 8, 19, 54, 5), 9598.0, 75919],
 [datetime.datetime(2022, 3, 8, 19, 54, 5), 9597.0, 75925],
 [datetime.datetime(2022, 3, 8, 19, 54, 5), 9597.0, 75926],
 [datetime.datetime(2022, 3, 8, 19, 54, 8), 9593.0, 75936],
 [datetime.datetime(2022, 3, 8, 19, 54, 10), 9580.0, 75963],
 [datetime.datetime(2022, 3, 8, 19, 54, 10), 9577.0, 75965],
 [datetime.datetime(2022, 3, 8, 19, 54, 12), 9584.0, 75968],
 [datetime.datetime(2022, 3, 8, 19, 54, 12), 9587.0, 75970],
 [datetime.datetime(2022, 3, 8, 

In [26]:
import pandas as pd
kite_func = Kite_functions(kite)

In [29]:
kite_func.convert_tick_data_to_candles(ltp_data, '1min')

,open,high,low,close,volume
date,,,,,
2022-03-08 19:54:00+05:30,9602.0,9604.0,9577.0,9583.0,160.0
2022-03-08 19:55:00+05:30,9591.0,9592.0,9564.0,9569.0,158.0
2022-03-08 19:56:00+05:30,9574.0,9598.0,9571.0,9597.0,150.0
2022-03-08 19:57:00+05:30,9594.0,9630.0,9594.0,9628.0,288.0
2022-03-08 19:58:00+05:30,9632.0,9644.0,9630.0,9632.0,114.0
2022-03-08 19:59:00+05:30,9653.0,9665.0,9643.0,9649.0,773.0
2022-03-08 20:00:00+05:30,9644.0,9652.0,9620.0,9635.0,469.0
2022-03-08 20:01:00+05:30,9640.0,9660.0,9640.0,9640.0,304.0
2022-03-08 20:02:00+05:30,9642.0,9647.0,9612.0,9619.0,302.0


In [10]:
#60751623
ticker.subscribe_list = [60751623]
ticker.subscribe_new_tokens = True

In [13]:
eval(r.get('60751623'+'_depth'))

{'buy': [{'quantity': 1, 'price': 545.1, 'orders': 1},
  {'quantity': 1, 'price': 545.0, 'orders': 1},
  {'quantity': 1, 'price': 542.0, 'orders': 1},
  {'quantity': 6, 'price': 540.0, 'orders': 3},
  {'quantity': 2, 'price': 538.5, 'orders': 1}],
 'sell': [{'quantity': 1, 'price': 549.9, 'orders': 1},
  {'quantity': 2, 'price': 551.6, 'orders': 1},
  {'quantity': 2, 'price': 551.9, 'orders': 1},
  {'quantity': 3, 'price': 552.0, 'orders': 2},
  {'quantity': 2, 'price': 552.7, 'orders': 1}]}

In [16]:
ticker.subscribe_tokens([58843655, 59736071])

AttributeError: 'NoneType' object has no attribute 'sendMessage'

In [8]:
ticker.unsubscribe_tokens([59736071, 58843655])

In [9]:
ticker.unsubscribe_list = [59736071]
ticker.unsubscribe_from_token = True

In [71]:
ticker.kws.unsubscribe([59736071, 58843655])

True

In [38]:
ticker.tokens.append(59736071)
ticker.tokens = [59736071, 58843655]

In [62]:
ticker.kws.subscribe([59736071, 58843655])
ticker.kws.set_mode(ticker.kws.MODE_FULL,ticker.tokens)

True

In [10]:
ticker.tokens

[58843655, 59736071]

In [12]:
eval(r.get('59736071'))

8392.0

In [13]:
eval(r.get('58843655'))

70359.0

In [10]:
eval(r.get('60751623'+'_depth'))

{'buy': [{'quantity': 1, 'price': 550.2, 'orders': 1},
  {'quantity': 2, 'price': 546.3, 'orders': 1},
  {'quantity': 1, 'price': 546.0, 'orders': 1},
  {'quantity': 1, 'price': 545.6, 'orders': 1},
  {'quantity': 4, 'price': 545.0, 'orders': 2}],
 'sell': [{'quantity': 2, 'price': 555.5, 'orders': 1},
  {'quantity': 2, 'price': 556.3, 'orders': 1},
  {'quantity': 2, 'price': 556.7, 'orders': 1},
  {'quantity': 1, 'price': 557.0, 'orders': 1},
  {'quantity': 1, 'price': 557.2, 'orders': 1}]}

In [11]:
ticker.subscribe_via_websocket([60751623])

In [19]:
ticker.start_ticker()

In [22]:
eval(r.get('60751623'))

458.0

In [13]:
ticker.websocket_is_open

True

In [21]:
eval(r.get(59736071))

8873.0

In [9]:
ticker.tokens.append(58843655)
ticker.start_ticker()

In [12]:
eval(r.get(58843655))

69604.0

In [15]:
eval(r.get('58843655'+'_depth'))

{'buy': [{'quantity': 1, 'price': 69595.0, 'orders': 1},
  {'quantity': 1, 'price': 69591.0, 'orders': 1},
  {'quantity': 1, 'price': 69575.0, 'orders': 1},
  {'quantity': 1, 'price': 69570.0, 'orders': 1},
  {'quantity': 1, 'price': 69569.0, 'orders': 1}],
 'sell': [{'quantity': 2, 'price': 69632.0, 'orders': 2},
  {'quantity': 1, 'price': 69642.0, 'orders': 1},
  {'quantity': 1, 'price': 69646.0, 'orders': 1},
  {'quantity': 1, 'price': 69659.0, 'orders': 1},
  {'quantity': 1, 'price': 69669.0, 'orders': 1}]}

In [16]:
ticker.unsubscribe_list = [58843655]
ticker.start_ticker()

Unsubscribing from tokens [58843655]


In [17]:
ticker.tokens

[59736071]

In [18]:
eval(r.get(58843655))

69600.0

In [77]:
ticker.tokens.append(60751623)
ticker.start_ticker()

In [103]:
eval(r.get(60751879))

800.0

In [104]:
eval(r.get('60751879'+'_depth'))

{'buy': [{'quantity': 2, 'price': 800.0, 'orders': 2},
  {'quantity': 11, 'price': 797.0, 'orders': 4},
  {'quantity': 3, 'price': 792.1, 'orders': 2},
  {'quantity': 1, 'price': 792.0, 'orders': 1},
  {'quantity': 3, 'price': 790.0, 'orders': 2}],
 'sell': [{'quantity': 1, 'price': 852.0, 'orders': 1},
  {'quantity': 1, 'price': 855.0, 'orders': 1},
  {'quantity': 1, 'price': 860.0, 'orders': 1},
  {'quantity': 1, 'price': 888.0, 'orders': 1},
  {'quantity': 1, 'price': 890.0, 'orders': 1}]}

In [98]:
ticker.tokens.append(60751879)
ticker.tokens

[59736071, 60751623, 60751623, 60751879]

In [99]:
ticker.unsubscribe_list = [60751879]
ticker.tokens.remove(60751879)
ticker.start_ticker()

Unsubscribing from tokens [60751879]


In [6]:
ticker.stop_ticker = True

In [20]:
ticker.start_ticker()

In [21]:
ticker.kws.close()

In [28]:
ticker.kws.stop()

Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


In [27]:
ticker.websocket_is_open, ticker.kws.is_connected()

(True, True)

In [10]:
type(datetime.datetime(2022, 1, 24, 10, 39, 49)) == datetime.datetime

True

In [11]:
datetime.datetime(2022, 1, 24, 10, 39, 49).hour

10

In [5]:
ticker.ticker_dict[18420482][-1]

[datetime.datetime(2022, 3, 7, 13, 14, 13), 177.0, 8356050]

In [15]:
ticker.ticker_dict[18420482][-1]

[datetime.datetime(2022, 3, 7, 13, 14, 31), 177.75, 8361000]

In [21]:
datetime.datetime.strptime('2022-03-07T12:48:57', '%Y-%m-%dT%H:%M:%S')

datetime.datetime(2022, 3, 7, 12, 48, 57)

In [16]:
kite_func.get_options_symbol_and_token('NIFTY', 15850, 'CE')

('NIFTY2231015850CE', 18419458)

In [17]:
ticker.ticker_dict[18419458][-1]

[datetime.datetime(2022, 3, 7, 13, 18, 43), 228.65, 11560200]

In [24]:
ticker.ticker_dict[18419458][-1]

[datetime.datetime(2022, 3, 7, 13, 18, 57), 232.05, 11563050]

In [37]:
ticker.depth_dict[18419458]

{'buy': [{'quantity': 50, 'price': 211.3, 'orders': 1},
  {'quantity': 250, 'price': 211.25, 'orders': 5},
  {'quantity': 200, 'price': 211.2, 'orders': 4},
  {'quantity': 100, 'price': 211.15, 'orders': 2},
  {'quantity': 50, 'price': 211.1, 'orders': 1}],
 'sell': [{'quantity': 50, 'price': 211.7, 'orders': 1},
  {'quantity': 450, 'price': 211.8, 'orders': 4},
  {'quantity': 450, 'price': 211.85, 'orders': 2},
  {'quantity': 100, 'price': 211.95, 'orders': 2},
  {'quantity': 150, 'price': 212.0, 'orders': 2}]}

In [5]:
ticker.websocket_is_open

True

In [6]:
ticker.ticker_dict[18419458]

[[datetime.datetime(2022, 3, 7, 14, 13, 3), 192.3, 13771600],
 [datetime.datetime(2022, 3, 7, 14, 13, 3), 191.4, 13771700],
 [datetime.datetime(2022, 3, 7, 14, 13, 4), 191.35, 13771700],
 [datetime.datetime(2022, 3, 7, 14, 13, 4), 191.55, 13772850],
 [datetime.datetime(2022, 3, 7, 14, 13, 5), 190.05, 13779500],
 [datetime.datetime(2022, 3, 7, 14, 13, 6), 189.75, 13777400],
 [datetime.datetime(2022, 3, 7, 14, 13, 6), 190.0, 13777400],
 [datetime.datetime(2022, 3, 7, 14, 13, 7), 190.0, 13781500],
 [datetime.datetime(2022, 3, 7, 14, 13, 8), 189.85, 13782600],
 [datetime.datetime(2022, 3, 7, 14, 13, 11), 189.25, 13782600],
 [datetime.datetime(2022, 3, 7, 14, 13, 11), 189.95, 13785950],
 [datetime.datetime(2022, 3, 7, 14, 13, 13), 189.3, 13786200],
 [datetime.datetime(2022, 3, 7, 14, 13, 14), 189.4, 13786250],
 [datetime.datetime(2022, 3, 7, 14, 13, 15), 189.8, 13792050],
 [datetime.datetime(2022, 3, 7, 14, 13, 15), 189.5, 13792850],
 [datetime.datetime(2022, 3, 7, 14, 13, 16), 190.15, 1379

In [14]:
import pandas as pd
df = pd.DataFrame(ticker.ticker_dict[18419458], columns=['ts','ltp', 'abs_volume'])
df

,ts,ltp,abs_volume
0,2022-03-07 14:43:50,229.35,15444450
1,2022-03-07 14:43:52,229.10,15444450
2,2022-03-07 14:43:52,228.30,15447550
3,2022-03-07 14:43:53,228.20,15447550
4,2022-03-07 14:43:53,228.55,15447700
...,...,...,...
3179,2022-03-07 15:29:57,236.90,18527500
3180,2022-03-07 15:29:58,232.40,18528100
3181,2022-03-07 15:29:58,232.45,18528100
3182,2022-03-07 15:29:59,237.80,18528400


In [15]:
df.shape

(3184, 3)

In [24]:
df['ts'] = df['ts'] + pd.to_timedelta(df.index,'us')
df

,ts,ltp,abs_volume
0,2022-03-07 14:13:03.000000,192.30,13771600
1,2022-03-07 14:13:03.000001,191.40,13771700
2,2022-03-07 14:13:04.000002,191.35,13771700
3,2022-03-07 14:13:04.000003,191.55,13772850
4,2022-03-07 14:13:05.000004,190.05,13779500
...,...,...,...
1762,2022-03-07 14:39:03.001762,225.90,15171800
1763,2022-03-07 14:39:04.001763,226.05,15171900
1764,2022-03-07 14:39:05.001764,227.00,15172500
1765,2022-03-07 14:39:05.001765,226.75,15173600


In [13]:
df = pd.DataFrame(ticker.ticker_dict[18419458], columns=['ts','ltp', 'abs_volume'])
df['ts'] = df['ts'] + pd.to_timedelta(df.index,'us')
data = df.copy()
data = data.set_index(['ts'])
data.index = pd.to_datetime(data.index)
ticks = data.loc[:, ['ltp']]   
dff=ticks['ltp'].resample('5min').ohlc()
dff['vol'] = data['abs_volume'].resample('5min').max().dropna()
dff['volume'] = dff.vol.diff()
dff

,open,high,low,close,vol,volume
ts,,,,,,
2022-03-07 14:40:00,229.35,236.75,228.20,235.40,15541500,NaN
2022-03-07 14:45:00,236.30,241.00,227.90,231.85,15902650,361150.0
2022-03-07 14:50:00,231.25,253.75,231.20,247.40,16363550,460900.0
2022-03-07 14:55:00,247.45,252.00,231.90,236.20,16672300,308750.0
2022-03-07 15:00:00,236.90,245.00,210.75,223.75,17078650,406350.0
2022-03-07 15:05:00,223.90,239.05,223.40,232.70,17369600,290950.0
2022-03-07 15:10:00,232.50,249.75,230.70,246.50,17755350,385750.0
2022-03-07 15:15:00,245.55,249.35,233.00,249.00,18102550,347200.0
2022-03-07 15:20:00,248.00,248.40,242.45,245.25,18311550,209000.0


In [26]:
dff

,open,high,low,close,vol,volume
ts,,,,,,
2022-03-07 14:13:00,192.30,196.45,189.25,196.45,13834650,NaN
2022-03-07 14:14:00,195.00,195.80,192.85,193.15,13888500,53850.0
2022-03-07 14:15:00,193.20,195.40,190.00,190.00,13935950,47450.0
2022-03-07 14:16:00,189.60,189.60,182.95,184.50,14035000,99050.0
2022-03-07 14:17:00,183.70,189.60,182.25,187.80,14120300,85300.0
2022-03-07 14:18:00,188.40,192.35,186.85,189.10,14163500,43200.0
2022-03-07 14:19:00,188.90,194.45,188.65,193.35,14192700,29200.0
2022-03-07 14:20:00,193.35,194.95,189.40,191.70,14223550,30850.0
2022-03-07 14:21:00,192.65,199.60,192.65,199.35,14259100,35550.0


In [18]:
kite_func.get_trading_symbol_from_token(18419458)

'NIFTY2231015850CE'

In [5]:
placed_order_id = kite.place_order(tradingsymbol='ACC',
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=1,
                            order_type=kite.ORDER_TYPE_LIMIT,
                            price=2100,
                            product=kite.PRODUCT_MIS,
                            variety=kite.VARIETY_AMO)

In [6]:
kite.ltp('NSE:ACC')['NSE:ACC']['last_price']

2122.65

In [10]:
telegram_bot_sendtext('testttttt ...')

In [12]:
telegram_bot_sendtext('Testing algo trader...')

In [13]:
telegram_bot_sendtext('Testing algo_trader...')